<a href="https://colab.research.google.com/github/Monsoon94/ADP/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%95%B8%EB%93%A4%EB%A7%81/%EA%B2%B0%EC%B8%A1%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

결측치 확인부터 처리까지의 내용 정리

# 결측치 확인

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "A":[1, 2, np.nan, 4],
    "B":[np.nan, 2, 3, 4],
    "C":["x", None, "y", "z"]
})

# True/False mask
df.isna()
df.isnull()# 동일한 기능

# 반대 (결측치가 아닌 값)
df.notna()
df.notnull()

# 결측치 개수 세기
df.isna().sum()      # 컬럼별 개수
df.isna().sum().sum()# 전체 개수

# 결측치 비율
df.isna().mean()
df.isna().sum()/len(df)

# 결측치 sort
df.isna().mean().sort_values(ascending=False)

,0
A,0.25
B,0.25
C,0.25


In [ ]:
# A 컬럼이 NaN인 행만
df[df["A"].isna()]

# B 컬럼이 NaN이 아닌 행만
df[df["B"].notna()]

# A 컬럼이 NaN이 아닌 행만
df[df["A"].notna()]

# 결측치 제거

In [ ]:
# 1) 결측치 포함된 행 제거
df.dropna()

# 2) 결측치 포함된 열 제거
df.dropna(axis=1)

# 3) 모든 값이 NaN인 행만 제거
df.dropna(how="all")

# 4) 특정 컬럼 기준으로 결측치 제거
df.dropna(subset=["A","B"])

# 결측치 채우기

In [ ]:
# 1) 상수로 대체
df.fillna(0)

# 2) 컬럼별 다른 값으로 대체
df.fillna({"A":0, "B":99})

# 3) 직전 값으로 채우기 (forward fill)
df.fillna(method="ffill")

# 4) 바로 다음 값으로 채우기 (backward fill)
df.fillna(method="bfill")

# 5) 평균/중앙값/최빈값으로 대체
df["A"].fillna(df["A"].mean())
df["A"].fillna(df["A"].median())
df["C"].fillna(df["C"].mode()[0])

# 응용

In [ ]:
# 결측치가 일정 비율 이상인 컬럼 제거
df = df.dropna(axis=1, thresh=len(df)*0.8)

# interpolate (수치형일 때 보간)
df["A"].interpolate(method="linear")

In [5]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "A":[1, 2, np.nan, np.nan],
    "B":[np.nan, np.nan, 3, 4],
})

print("Before")
display(df)

# A에 결측치 채울 때 B 활용하여 채우기
df['A'] = df["A"].fillna(df["B"])

print("After")
display(df)

Before


,A,B
0,1.0,NaN
1,2.0,NaN
2,NaN,3.0
3,NaN,4.0


After


,A,B
0,1.0,NaN
1,2.0,NaN
2,3.0,3.0
3,4.0,4.0


# 시계열 데이터 결측치

## 시간(인덱스)이 결측인 경우

## 값이 결측인 경우

In [22]:
import pandas as pd

s = pd.Series([None, 1, None, 3, None, None, None, 6, 7, 8, 9, None])

# numpy array 형태
# print(s.values)

# 파이썬 list 형태
print(s.to_list())

[nan, 1.0, nan, 3.0, nan, nan, nan, 6.0, 7.0, 8.0, 9.0, nan]


In [31]:
print('original : ', s.to_list())

# ffill
s_ffill = s.ffill()
print("ffill : ", s_ffill.to_list())

# bfill
s_bfill = s.bfill()
print("bfill : ", s_bfill.to_list())

#
s_bf_ff = s.bfill().ffill()
print("bf_ff : ", s_bf_ff.to_list())

original :  [nan, 1.0, nan, 3.0, nan, nan, nan, 6.0, 7.0, 8.0, 9.0, nan]
ffill :  [nan, 1.0, 1.0, 3.0, 3.0, 3.0, 3.0, 6.0, 7.0, 8.0, 9.0, 9.0]
bfill :  [1.0, 1.0, 3.0, 3.0, 6.0, 6.0, 6.0, 6.0, 7.0, 8.0, 9.0, nan]
bf_ff :  [1.0, 1.0, 3.0, 3.0, 6.0, 6.0, 6.0, 6.0, 7.0, 8.0, 9.0, 9.0]


In [52]:
# 좀 더 디테일하게 결측치 채우기
import numpy as np
import pandas as pd

s = pd.Series([None, 1, None, 3, None, None, None, 6, 7, 8, 9, None])
print('original : ', s.to_list())

# 조건 1. 결측치가 존재하는 데이터의 직전 값과 다음 값의 평균으로 대치한다.
# 조건 2. 연속된 결측치가 있다면 결측치가 아닌 이전 시간대의 값과 이후 시간대의 값의 평균으로 모든 연속 시간대 결측치를 대치한다.
# 조건 3. 맨앞이나 맨뒤에 결측치가 존재한다면 직전, 직후 시간대 값으로 대치한다.

s_sol = pd.concat([s.ffill(), s.bfill()]).groupby(level=0).mean()
print('sol :      ', s_sol.to_list())

original :  [nan, 1.0, nan, 3.0, nan, nan, nan, 6.0, 7.0, 8.0, 9.0, nan]
sol :       [1.0, 1.0, 2.0, 3.0, 4.5, 4.5, 4.5, 6.0, 7.0, 8.0, 9.0, 9.0]
